In [1]:
import os
import argparse
import time
from datetime import datetime, date
import random

import numpy as np
from scipy.sparse import load_npz
from scipy.stats import pearsonr
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import pandas as pd

import torch
import torch_geometric
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from model_classes_ import GCN_classification, GCN_regression
from custom_funcs import experiment_n

In [2]:
torch.cuda.empty_cache()

In [3]:
print(torch.cuda.is_available())  # Should print `True`
print(torch.cuda.device_count())  # Should print the number of GPUs available
print(torch.cuda.current_device())  # Should print the current GPU ID (e.g., `0`)
print(torch.cuda.get_device_name(0))  # Should print the GPU name

True
1
0
NVIDIA GeForce RTX 2060


# Data Preparation

In [4]:
# Hyperparameters

chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)
regression_flag = 0
max_epoch = 50
learning_rate = 0.001
num_lin_layers = 2
lin_hidden_size = 100
num_graph_conv_layers = 2
graph_conv_embed_size = 256
num_runs = 10
graph_conv_layer_sizes = [num_feat] + \
        [int(max(graph_conv_embed_size, lin_hidden_size)) \
              for i in np.arange(1, num_graph_conv_layers, 1)] + [lin_hidden_size]

lin_hidden_sizes_r = [graph_conv_layer_sizes[-1]] + \
        [int(max(lin_hidden_size, 1)) \
              for i in np.arange(1, num_lin_layers, 1)] + [1]
lin_hidden_sizes_c = [graph_conv_layer_sizes[-1]] + \
        [int(max(lin_hidden_size, 2)) \
              for i in np.arange(1, num_lin_layers, 1)] + [2]

In [5]:
# Stores AUROC and PCC across all models for each cell line
E116 = {'name': 'E116', 'AUROC': None, 'PCC': None}
E122 = {'name': 'E122', 'AUROC': None, 'PCC': None}
E123 = {'name': 'E123', 'AUROC': None, 'PCC': None}

In [6]:
def experimentCNN(model, cell_line, regression_flag, optimizer, criterion, num_epochs=10):
    if regression_flag == 1:
        task = 'Regression'
    else:
        task = 'Classification'
    
    print(f"Task: {task}")
    # Create all paths
    base_path = os.getcwd()
    hic_sparse_mat_file = os.path.join(base_path, 'data', cell_line['name'], 'hic_sparse.npz')
    np_hmods_norm_all_file = os.path.join(base_path, 'data', cell_line['name'], \
        'np_hmods_norm_chip_' + str(chip_res) + 'bp.npy')
    np_nodes_lab_genes_file = os.path.join(base_path, 'data',  cell_line['name'], \
        'np_nodes_lab_genes_reg' + str(regression_flag) + '.npy')
    
    allNodes_hms = np.load(np_hmods_norm_all_file)
    hms = allNodes_hms[:, 1:] # only includes features, not node ids
    X = torch.tensor(hms).float().reshape(-1, num_feat) # shape: [279606, 6]
    allNodes = allNodes_hms[:, 0].astype(int) # shape: [279606,1]
    
    geneNodes_labs = np.load(np_nodes_lab_genes_file)
    geneNodes = geneNodes_labs[:, -2].astype(int) # shape: [16699,1]
    allLabs = -1*np.ones(np.shape(allNodes))
    
    targetNode_mask = torch.tensor(geneNodes).long()

    # Generate y labels
    geneLabs = geneNodes_labs[:, -1].astype(int)
    allLabs[geneNodes] = geneLabs
    Y = torch.tensor(allLabs).long() # shape: [279606,1]

    # Shuffle indices
    pred_idx_shuff = torch.randperm(targetNode_mask.shape[0])
    
    fin_train = np.floor(0.7*pred_idx_shuff.shape[0]).astype(int)
    fin_valid = np.floor(0.85*pred_idx_shuff.shape[0]).astype(int)
    train_idx = pred_idx_shuff[:fin_train]
    val_idx = pred_idx_shuff[fin_train:fin_valid]
    test_idx = pred_idx_shuff[fin_valid:]

    # Generate train, validation, and test sets
    train_data = X[targetNode_mask][train_idx]
    val_data = X[targetNode_mask][val_idx]
    test_data = X[targetNode_mask][test_idx]
    

    if not regression_flag:
        train_labels = torch.tensor(geneNodes_labs[train_idx][:,1]).long()
        val_labels = torch.tensor(geneNodes_labs[val_idx][:,1]).long()
        test_labels = torch.tensor(geneNodes_labs[test_idx][:,1]).long()
    else:
        train_labels = torch.tensor(geneNodes_labs[train_idx][:,1]).float()
        val_labels = torch.tensor(geneNodes_labs[val_idx][:,1]).float()
        test_labels = torch.tensor(geneNodes_labs[test_idx][:,1]).float()
        

    # Reformat data
    train_data = train_data.unsqueeze(1)
    val_data = val_data.unsqueeze(1)
    test_data = test_data.unsqueeze(1)

    # Create data loaders
    BATCH_SIZE = 32
    
    print("Create data loaders")
    train_dataset = TensorDataset(train_data, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    val_dataset = TensorDataset(val_data, val_labels)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    test_dataset = TensorDataset(test_data, test_labels)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
    print("Data loaders created")
    print(f"Train loader shape: {len(train_loader)}")
    print(f"Val loader shape: {len(val_loader)}")
    print(f"Test loader shape: {len(test_loader)}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    print(f"Device: {device}")

    # Train data
    print("Begin training + evaluation...\n\n")
    metric_scores = []
    for epoch in range(num_epochs):
        start = time.time()
        epoch_loss = 0
        for batch_inputs, batch_labels in train_loader:
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)
            optimizer.zero_grad()
            
            output = model(batch_inputs)
            if regression_flag:
                batch_labels = batch_labels.view(-1, 1)
                
            loss = criterion(output, batch_labels)
            
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            
        end = time.time()
        avg_loss = epoch_loss / len(train_loader)
        print(f"Training time: {end-start} seconds")
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}\n")

        model.eval()
        test_outputs = []
        test_targets = []
        start = time.time()
        with torch.no_grad():
            for batch_inputs, batch_labels in test_loader:
                batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)
                output = model(batch_inputs).squeeze()
                test_outputs.append(output)
                test_targets.append(batch_labels)
        
        test_outputs = torch.cat(test_outputs).cpu().numpy()
        test_targets = torch.cat(test_targets).cpu().numpy()
        
        end = time.time()
        print(f"Testing time: {end-start} seconds")
        if regression_flag == 0:
            test_outputs = test_outputs[:, 1]
            auroc = roc_auc_score(test_targets, test_outputs)
            print(f"Epoch {epoch + 1}, AUROC Score: {auroc:.4f}")
            metric_scores.append(auroc)
        else:
            # Calculate PCC for regression
            pcc, _ = pearsonr(test_targets, test_outputs)
            print(f"Epoch {epoch + 1}, PCC Score: {pcc:.4f}")
            metric_scores.append(pcc)
    
    metric_scores = np.array(metric_scores)
    if regression_flag == 0:
        cell_line['AUROC'] = (np.mean(metric_scores), np.std(metric_scores))
    else:
        cell_line['PCC'] = (np.mean(metric_scores), np.std(metric_scores))

# Model

In [7]:
class CNN(nn.Module):
    def __init__(self, regression_flag=0):
        super(CNN, self).__init__()

        INPUT_LENGTH = 6
        NUM_CLASSES = 2 if not regression_flag else 1
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(16 * INPUT_LENGTH, 32)
        self.fc2 = nn.Linear(32, NUM_CLASSES)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
            x = self.relu(self.conv1(x))
            x = x.view(x.size(0), -1)         
            x = self.relu(self.fc1(x))
            x = self.fc2(x)  
            return x

# Model Training And Evaluation

In [8]:
for regression_flag in [0,1]:
    for cell_line in [E116,E122,E123]:
        print(f"Cell line: {cell_line['name']}")
        model = CNN(regression_flag=regression_flag)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.MSELoss() if regression_flag else nn.CrossEntropyLoss()
        experimentCNN(model, cell_line, regression_flag, optimizer, criterion)

Cell line: E116
Task: Classification
Create data loaders
Data loaders created
Train loader shape: 366
Val loader shape: 79
Test loader shape: 79
Device: cuda
Begin training + evaluation...


Training time: 4.00205659866333 seconds
Epoch 1/10, Loss: 0.4761

Testing time: 0.06755900382995605 seconds
Epoch 1, AUROC Score: 0.9011
Training time: 1.0829346179962158 seconds
Epoch 2/10, Loss: 0.3887

Testing time: 0.08206748962402344 seconds
Epoch 2, AUROC Score: 0.9044
Training time: 1.0981354713439941 seconds
Epoch 3/10, Loss: 0.3826

Testing time: 0.08244919776916504 seconds
Epoch 3, AUROC Score: 0.9058
Training time: 1.1246511936187744 seconds
Epoch 4/10, Loss: 0.3807

Testing time: 0.08772921562194824 seconds
Epoch 4, AUROC Score: 0.9069
Training time: 1.1679527759552002 seconds
Epoch 5/10, Loss: 0.3799

Testing time: 0.08462119102478027 seconds
Epoch 5, AUROC Score: 0.9076
Training time: 1.28035569190979 seconds
Epoch 6/10, Loss: 0.3780

Testing time: 0.07624936103820801 seconds
Epoch 6,

In [9]:
print(f'E116:{E116}')
print(f'E122:{E122}')
print(f'E123:{E123}')

E116:{'name': 'E116', 'AUROC': (0.9068851266092259, 0.0024064489225907944), 'PCC': (0.7749560475640648, 0.007025365026968033)}
E122:{'name': 'E122', 'AUROC': (0.8905679597655123, 0.0029963894268450007), 'PCC': (0.7233833498765103, 0.01536840043941363)}
E123:{'name': 'E123', 'AUROC': (0.9267664737702453, 0.0028554446257380408), 'PCC': (0.8016334751712677, 0.011083299512114153)}


In [10]:
with open('results/CNN.csv', 'w') as f:
    f.write('cell_line,auroc_mu,auroc_std,pcc_mu,pcc_std\n')
    f.write(f"E116,{E116['AUROC'][0]},{E116['AUROC'][1]},{E116['PCC'][0]},{E116['PCC'][1]}\n")
    f.write(f"E122,{E122['AUROC'][0]},{E122['AUROC'][1]},{E122['PCC'][0]},{E122['PCC'][1]}\n")
    f.write(f"E123,{E123['AUROC'][0]},{E123['AUROC'][1]},{E123['PCC'][0]},{E123['PCC'][1]}\n")

f.close()